In [ ]:
"""
 - no merging APs - separate APs work better 
 - Average works best (99% 1 step + 2 steps, pixel) (98% 1 step + 2 steps, redmi)
 - weighting by power used / total power received doesn't work 
 - KS doesnt work 
 - Average > Median > Max 
 - random direction = disaster 
 - sometimes RSSI data from an AP is missing (not received) -> misclassification
"""

In [1]:
from utils import *
import sys, os, random
# from compare_locations import compare_locations

datadir = "./data/nowl/"
# preprocessed already, with 


In [2]:
pix_etaje = [
"0-pixel-04-03-2021_15-32-38.json",
"1-pixel-25-02-2021_21-16-46.json",
"2-pixel-25-02-2021_16-27-45.json",
"3-pixel-04-03-2021_16-18-50.json",
"4-pixel-04-03-2021_16-43-50.json",
"5-pixel-04-03-2021_17-11-32.json",
"6-pixel-17-03-2021_12-52-29.json",
"7-pixel-17-03-2021_13-14-36.json"]

red_etaje = [
"0-redmi-04-03-2021_15-31-12.json",
"1-redmi-25-02-2021_21-16-10.json",
"2-redmi-25-02-2021_16-27-08.json",
"3-redmi-04-03-2021_16-18-52.json",
"4-redmi-04-03-2021_16-43-45.json",
"5-redmi-04-03-2021_17-11-30.json",
"6-redmi-17-03-2021_12-52-29.json",
"7-redmi-17-03-2021_13-14-39.json"]


retaje = []
for e in red_etaje: 
    json_file = os.path.join(datadir, e)
    etaj = load_dataset_json(json_file)
    retaje.append(etaj)

petaje = []
for e in pix_etaje: 
    json_file = os.path.join(datadir, e)
    etaj = load_dataset_json(json_file)
    petaje.append(etaj)
    

In [3]:
import numpy as np
from scipy.spatial.distance import braycurtis
from scipy.stats import ks_2samp, chisquare
import copy 

def merge_wifi_fingerprints(flist):
    fingerprint = copy.deepcopy(flist[0])
    for f2 in copy.deepcopy(flist[1:]):
        if not "wifi" in f2.keys():
            continue
        for mac in f2["wifi"].keys():
            if not mac in fingerprint["wifi"].keys():
                fingerprint["wifi"][mac] = f2["wifi"][mac]
                #fingerprint["wifi"][mac]['rssi'] = []
            else:
                fingerprint["wifi"][mac]['rssi'].extend(f2["wifi"][mac]['rssi'])
            fingerprint["wifi"][mac]['rssi'].sort()    
    return fingerprint


def compare_locations(c1, c2, simil_method = braycurtis,  selection = 'Average', dif = False, direction=4):
    rssi1 = []
    rssi2 = []
    w = [] #weight for braycurtis
    
    if (direction == 1):
        # take the first fingerprint in the list 
        wifi1 = c1['fingerprints'][0]['wifi']
        wifi2 = c2['fingerprints'][0]['wifi']

        # take a random fingerprint in the list 
        #wifi1 = c1['fingerprints'][random.randint(0,3)]['wifi']
        #wifi2 = c2['fingerprints'][random.randint(0,3)]['wifi']

    if (direction == 2) :   
        # first & third - as if measuring twice at 180 degrees
        wifi1 = merge_wifi_fingerprints([c1['fingerprints'][0], c1['fingerprints'][2]])['wifi']
        wifi2 = merge_wifi_fingerprints([c2['fingerprints'][0], c2['fingerprints'][2]])['wifi']
    
    if (direction == 4):  
        #merge all existing fingerprints for each collection
        wifi1 = merge_wifi_fingerprints(c1['fingerprints'])['wifi']
        wifi2 = merge_wifi_fingerprints(c2['fingerprints'])['wifi']
    
    # all power received by the query
    power1 = 0
    for ap in wifi1:
        power1 += np.sum(adjust_rssi(wifi1[ap]['rssi']))
        
    common_aps = list(set(wifi1.keys()) & set(wifi2.keys()))
    #print("common aps=", len(common_aps))
    
    # No APs in common -> similarity = 1
    if not common_aps:
        return 1

    # TODO: find the best metric
    # If not enough common APs -> similarity = 1
    if len(common_aps) * 10 < len(wifi1.keys()) or len(common_aps) < 3:
        return 1

    for ap in common_aps:
        
        if selection == 'Bestdir':
            for f1 in c1['fingerprints']: # all fingerprint dirs in the query 
                if not ap in f1['wifi'].keys():
                    continue
                m1 = np.average(adjust_rssi(f1['wifi'][ap]['rssi']))
                best = 100                
                for f2 in c2['fingerprints']: 
                    if not ap in f2['wifi'].keys():
                        continue
                    m2 = np.average(adjust_rssi(f2['wifi'][ap]['rssi']))
                    if(math.fabs(m2-m1)) < best:  # find one dir with closest RSSI
                        best = math.fabs(m2-m1)
                        bestf = m2
                rssi1.append(m1)
                rssi2.append(bestf)
                
        # Take only the first RSSI value                
        if selection == 'First':
            rssi1.append(wifi1[ap]['rssi'][0])
            rssi2.append(wifi2[ap]['rssi'][0])

        # Make an average of all RSSI values
        if selection == 'Average':
            rssi1.append(np.average(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.average(adjust_rssi(wifi2[ap]['rssi'])))
            w.append(min(len(wifi1[ap]['rssi']), len(wifi2[ap]['rssi'])))

        if selection == 'Median':
            rssi1.append(np.median(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.median(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Mean':
            rssi1.append(np.mean(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.mean(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Std':
            rssi1.append(np.std(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.std(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Max':
            rssi1.append(np.max(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.max(adjust_rssi(wifi2[ap]['rssi'])))
            
        if selection == 'KS':
            _, p = ks_2samp(wifi1[ap]['rssi'], wifi2[ap]['rssi'])
            rssi1.append(p)
            rssi2.append(1.0)

        if selection == 'Tempered':
            rss_1.append(np.average(rssi_v[index][key]) * random.uniform(0.8, 1.2))
            rss_2.append(np.average(rssi_v[r][key]) * random.uniform(0.8, 1.2))
            
    rssi11 = []
    rssi22 = []
    if selection != 'KS':
        
        if (dif == True):
            #print(rssi1)
            rssi11.append(np.diff(rssi1))
            rssi22.append(np.diff(rssi2))
            rssi1 = rssi11
            rssi2 = rssi22        

    #print(f"1={rssi1}\n2={rssi2}\ncommon={common_aps}")
    #if selection != 'KS':
    #    rssi1 = list(map(lambda x: x - np.mean(rssi1), rssi1))
    #    rssi2 = list(map(lambda x: x - np.mean(rssi2), rssi2))
    return simil_method(tuple(rssi1), tuple(rssi2))
#        return simil_method(tuple(rssi1), tuple(rssi2))*sum(rssi1)/power1
# weigh final result with used power/total power   

In [4]:
# Generate a matrix (list of lists) for a collection (etaj)
# [point_index, index_of_closest_pt, dist_to_closest_pt, index_diff, dist_2_best_neighbor]
def test_allvsall(collection, selection='Average', dif = False, direction = 4):
    lc = len(collection)
    #arr = [[0]*lc]*lc - nu merge!
    arr = [[0 for i in range(lc)] for j in range(lc)]
    for l in range(0, lc):
        for c in range(0, l):
            arr[l][c] = compare_locations(collection[l], collection[c], braycurtis, selection, dif, direction)
            arr[c][l] = arr[l][c]
        arr[l][l] = 1 # max 

   # print("#pt minpt mindist diff dist2nbr")
    listp = []
    for l in range(0, lc):
        mindist = min(arr[l])
        index_of_min = arr[l].index(mindist)
        diff = diff_modulo(l, arr[l].index(mindist), lc)
        distl=arr[l][(l-1+lc)%lc]
        distr=arr[l][(l+1)%lc]

        #print(l, index_of_min, mindist, diff, min(distl, distr))
        listp.append([l, index_of_min, mindist, diff, min(distl, distr)])
    return listp



def test_queryvsall(query, collections, selection='Average', dif = False, direction = 4):
    """
    query = collection, an array or fingerprints 
    collections = aray of collection, can be a floor  
    RETURNS (similarity, index, eu_distance)
    """
    min = 1.0 
    mini = -1; 
    for c in range(0, len(collections)):
        d = compare_locations(query, collections[c], braycurtis, selection, dif, direction)
        if d != 0 and d < min:
            min = d 
            mini = c
    return min,mini, \
            euclidean([query['x'], query['y'], query['z']],\
                       [collections[mini]['x'], collections[mini]['y'],collections[mini]['z']])

  
    

In [ ]:
#  query = Pixel  
#  DB = redmi 
cl_sim = []
cl_steps = []
cl_dist = []

cl_sim_dif = []
cl_steps_dif = []
cl_dist_dif = []

for e in range(0, len(retaje)):
#for e in range(0, len(petaje)): 
    for p in range(0, len(petaje[e])):
        sim, index, d =  test_queryvsall(petaje[e][p], retaje[e], 'Average', dif = False, direction = 4)
        # print(sim, diff_modulo(p, index, len(petaje[e])), d)
        cl_sim.append(sim)
        cl_steps.append(diff_modulo(p, index, len(petaje[e])))
        cl_dist.append(d)
        
        sim_dif, index_dif, d_dif =  test_queryvsall(petaje[e][p], retaje[e], 'Average', dif = True, direction = 4)
        # print(sim, diff_modulo(p, index, len(petaje[e])), d)
        cl_sim_dif.append(sim_dif)
        cl_steps_dif.append(diff_modulo(p, index_dif, len(petaje[e])))
        cl_dist_dif.append(d_dif)
   

In [ ]:

b = np.arange(0, 0.3, 0.01) # Bins of histogram - from 1 to 5 
bin_width = b[1] - b[0]
hist_sim, bins_out = np.histogram(np.array(cl_sim), b, density=False)
hist_sim = hist_sim / len(cl_sim)
hist_sim_dif, bins_out_dif = np.histogram(np.array(cl_sim_dif), b, density=False)
hist_sim_dif = hist_sim_dif / len(cl_sim_dif)
fig = plt.figure()
plt.bar(b[:-1], hist_sim, width=bin_width, ec='k', alpha=0.5, label='avg')
plt.bar(b[:-1], hist_sim_dif, width=bin_width, ec='k', alpha=0.5, label='avg dif')
plt.xlabel("Similarity closest point query=Pixel DB=Redmi")
plt.ylabel("PMF")
plt.title("{} {} \n{}{}".format("TOTAL floors ", len(retaje), len(cl_sim), " points;"))
plt.tight_layout()
plt.legend(loc="upper right",shadow=True, fancybox=True)
plt.show()
#    fig.savefig(f"{method}-{len(etaje)}.pdf", bbox_inches='tight')
#    print("Percentages: ", hist_away)

b = np.arange(1, 15, 1) # Bins of histogram - from 1 to 5 
bin_width = b[1] - b[0]
hist_sim, bins_out = np.histogram(np.array(cl_steps), b, density=False)
hist_sim = hist_sim / len(cl_sim)
hist_sim_dif, bins_out_dif = np.histogram(np.array(cl_steps_dif), b, density=False)
hist_sim_dif = hist_sim_dif / len(cl_sim_dif)
fig = plt.figure()
plt.bar(b[:-1], hist_sim, width=bin_width, ec='k', alpha=0.5, label='avg')
plt.bar(b[:-1], hist_sim_dif, width=bin_width, ec='k', alpha=0.5, label='avg dif')
plt.xlabel("closest point (steps) query=Pixel DB=Redmi")
plt.ylabel("PMF")
plt.title("{} {} \n{}{}".format("TOTAL floors ", len(retaje), len(cl_sim), " points;"))
plt.tight_layout()
plt.legend(loc="upper right",shadow=True, fancybox=True)
plt.show()
#fig.savefig(f"{method}-{len(etaje)}.pdf", bbox_inches='tight')
#    print("Percentages: ", hist_away)



b = np.arange(0, 10, 1) # Bins of histogram - from 1 to 5 
bin_width = b[1] - b[0]
hist_sim, bins_out = np.histogram(np.array(cl_dist), b, density=False)
hist_sim = hist_sim / len(cl_sim)
hist_sim_dif, bins_out_dif = np.histogram(np.array(cl_dist_dif), b, density=False)
hist_sim_dif = hist_sim_dif / len(cl_sim_dif)
fig = plt.figure()
plt.bar(b[:-1], hist_sim, width=bin_width, ec='k', alpha=0.5, label='avg')
plt.bar(b[:-1], hist_sim_dif, width=bin_width, ec='k', alpha=0.5, label='avg dif')
plt.xlabel("closest point (meters) query=Pixel DB=Redmi")
plt.ylabel("PMF")
plt.title("{} {} \n{} {} \n{}{}".format("Meters: median of avg", np.median(cl_dist), "median of avg dif", np.median(cl_dist_dif), len(cl_sim), " points;"))
plt.tight_layout()
plt.legend(loc="upper right",shadow=True, fancybox=True)
plt.show()
#    fig.savefig(f"{method}-{len(etaje)}.pdf", bbox_inches='tight')
#    print("Percentages: ", hist_away)




cdf=np.cumsum(hist_sim)
cdf_dif=np.cumsum(hist_sim_dif)
fig = plt.figure()
plt.axhline(y=1, color='r', linestyle='--')
plt.xlabel("closest point (meters) query=Pixel DB=Redmi")
plt.ylabel("CDF(%)")
plt.title("{}".format("CDF - 4 directions - raw vs. difference"))
plt.plot(b[:-1], cdf, "x-", label='avg')
plt.plot(b[:-1], cdf_dif, "o-", label='avg dif')
plt.grid(True)
plt.tight_layout()
plt.legend(loc="upper right",shadow=True, fancybox=True)
plt.show()
fig.savefig("4directionAvgCDF.pdf", bbox_inches='tight')

